*****
After gathering scoreboard data, I can get insights into team compositions.   

There have been difficulties with scraping this information, even with the use of the API. This is because websites like tracker.gg employ Cloudfare protection, which means that they actively work against people scraping their data.     

So, since I frequently get rate limited and i have to take breaks/timeouts before attempting again, I have been able to get about 3,666 matches but not the full 5,400, after 15+hrs of scraping. This is not the ideal scenario but it is enough data to still work with.    

I think that in the future, longer periods of time could be allotted to scraping, but since the possibility of being able to scrape this data was not discovered until recently, it is reasonable to work with what I have gotten so far. (upon inspection of other data, it was revealed the internal match ids used by tracker.gg, which is appended at the end of each api call to the scoreboard, which i did not have a way of knowing until this)
*****

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('full_matches.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(tables)
#conn.close()
#matches.head()

[('full_matches',)]


In [2]:
matches = pd.read_sql_query("SELECT * FROM full_matches", conn)
conn.close()
matches.head()

,match_id,raw_json
0,5513730_1765074019_1318548_11001_12,"{""data"": {""attributes"": {""id"": ""5513730_176507..."
1,6711722_1764888815_1272257_11001_22,"{""data"": {""attributes"": {""id"": ""6711722_176488..."
2,6711647_1764886525_1245250_11001_22,"{""data"": {""attributes"": {""id"": ""6711647_176488..."
3,506241_1764884213_1311009_11001_21,"{""data"": {""attributes"": {""id"": ""506241_1764884..."
4,506105_1764882596_1310017_11001_21,"{""data"": {""attributes"": {""id"": ""506105_1764882..."


*****
It appears the same kind of raw json formatting is present as when i unpacked the file in the 'exploratory_analysis.ipynb" notebook. As so, i am going to copy over that code here to do the same thing. 
*****

In [3]:
import json 
matches['raw_json'] = matches['raw_json'].apply(json.loads)
matches

,match_id,raw_json
0,5513730_1765074019_1318548_11001_12,{'data': {'attributes': {'id': '5513730_176507...
1,6711722_1764888815_1272257_11001_22,{'data': {'attributes': {'id': '6711722_176488...
2,6711647_1764886525_1245250_11001_22,{'data': {'attributes': {'id': '6711647_176488...
3,506241_1764884213_1311009_11001_21,{'data': {'attributes': {'id': '506241_1764884...
4,506105_1764882596_1310017_11001_21,{'data': {'attributes': {'id': '506105_1764882...
...,...,...
5413,5517996_1765106140_1311312_11001_11,{'data': {'attributes': {'id': '5517996_176510...
5414,5517410_1765104631_1291702_11001_11,{'data': {'attributes': {'id': '5517410_176510...
5415,5517426_1765100984_1267184_11001_11,{'data': {'attributes': {'id': '5517426_176510...
5416,5513998_1765096434_1291511_11001_12,{'data': {'attributes': {'id': '5513998_176509...


In [4]:
print(json.dumps(matches["raw_json"].iloc[0], indent=4))

{
    "data": {
        "attributes": {
            "id": "5513730_1765074019_1318548_11001_12",
            "mode": "competitive",
            "mapId": 1318
        },
        "metadata": {
            "mapName": "Celestial Husk",
            "mapModeName": "Domination",
            "mapImageUrl": "https://trackercdn.com/cdn/tracker.gg/marvel-rivals/images/maps/hero/1318.jpg?v=1756050380",
            "timestamp": "2025-12-07T02:30:35+00:00",
            "duration": 381,
            "modeName": "Competitive",
            "isRanked": true,
            "winningTeamId": 0,
            "scores": [
                2,
                0
            ],
            "replayId": "10858083708",
            "fullMatchAvailable": true
        },
        "segments": [
            {
                "type": "player",
                "attributes": {
                    "accountId": "574bec15-67b9-412e-b988-3634e5a88915"
                },
                "metadata": {
                    "platformInfo"

In [5]:

rows = []

for i, row in matches.iterrows():
    raw_json = row["raw_json"]

    # Skip any missing data w/ continue
    if raw_json is None or "data" not in raw_json:
        continue
    
    data = raw_json["data"]
    match_id = data["attributes"]["id"]

    match_meta = data.get("metadata", {})
    match_map = match_meta.get("mapName")
    match_mode = match_meta.get("mapModeName")
    match_duration = match_meta.get("duration")
    winning_team = match_meta.get("winningTeamId")

    # need to go through all segments to get just the player stats 
    #that i want so i can match with the player data i already have
    for segment in data.get("segments", []):
        if segment.get("type") != "player": #theres data with non player data that idc about so skip
            continue  #skip it


    #originally i did segment['metadata'] bc cGPT said to do so
    # but added get() instead to avoid key errors 
    # so the loop could continue without crashing even if some data is missing

        meta = segment.get("metadata", {})
        stats = segment.get("stats", {})

        player_id = meta.get("platformInfo", {}).get("platformUserIdentifier")
        team_id = meta.get("teamId")
        result = meta.get("result")   #result is win result
        is_mvp = meta.get("isMvp", False) #assume false if there isnt the field there
        is_svp = meta.get("isSvp", False) #assume false if there isnt the field there

        # hero played
        heroes = meta.get("heroes", [])
        hero_name = heroes[0].get("name") if heroes else None
        hero_id = heroes[0].get("heroId") if heroes else None

        # flatten stats like in the other notebook
        flat_stats = {}
        for stat_key, stat in stats.items():
            flat_stats[f"{stat_key}_value"] = stat.get("value")

        rows.append({
            "match_id": match_id,
            "player_id": player_id,
            "team_id": team_id,
            "won": 1 if team_id == winning_team else 0, #need to distinguish meaning of 1/0 with w/l
            "hero_name": hero_name,
            "hero_id": hero_id,
            "map_name": match_map,
            "match_mode": match_mode,
            "duration_sec": match_duration,
            "is_mvp": is_mvp,
            "is_svp": is_svp,
            **flat_stats
        })

matches_cleaned = pd.DataFrame(rows)


In [6]:
matches_cleaned

,match_id,player_id,team_id,won,hero_name,hero_id,map_name,match_mode,duration_sec,is_mvp,...,featureHitRate2ChaosHits_value,featureHitRate2AllyHits_value,featureHitRate2HeroHits_value,featureHitRate2EnemyHits_value,featureHitRate2ShieldHits_value,featureHitRate2SummonerHits_value,featureSpecialData1Total_value,featureSpecialData1Value_value,ranked_value,rankedDelta_value
0,5513730_1765074019_1318548_11001_12,Deleted404,0,1,Rocket Raccoon,1023,Celestial Husk,Domination,381,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5025.0,22.0
1,5513730_1765074019_1318548_11001_12,Kaifs,0,1,Magneto,1037,Celestial Husk,Domination,381,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5057.0,20.0
2,5513730_1765074019_1318548_11001_12,Finnsi,0,1,Groot,1027,Celestial Husk,Domination,381,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5156.0,17.0
3,5513730_1765074019_1318548_11001_12,JukeBox2274,1,0,Angela,1056,Celestial Husk,Domination,381,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4976.0,-19.0
4,5513730_1765074019_1318548_11001_12,VP Sypeh,0,1,Invisible Woman,1050,Celestial Husk,Domination,381,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5308.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64051,5513925_1765094719_1231224_11001_12,iffvy,1,0,Winter Soldier,1041,Yggdrasill Path,Convoy,1059,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4983.0,-20.0
64052,5513925_1765094719_1231224_11001_12,Rajahyrax,1,0,Groot,1027,Yggdrasill Path,Convoy,1059,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4932.0,-20.0
64053,5513925_1765094719_1231224_11001_12,Пyк,1,0,Rocket Raccoon,1023,Yggdrasill Path,Convoy,1059,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4949.0,-20.0
64054,5513925_1765094719_1231224_11001_12,FAZE BLAZIKEN,0,1,Phoenix,1054,Yggdrasill Path,Convoy,1059,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5013.0,18.0


*****
This row number is fine:  

52,524 rows / 12 players/game = 4,377 games worth of data 
*****

will do the grouping by match id when analyzing data, just how player_id was used to group data in player data notebook

In [7]:
matches_cleaned[matches_cleaned.player_id == "VP Sypeh"]


,match_id,player_id,team_id,won,hero_name,hero_id,map_name,match_mode,duration_sec,is_mvp,...,featureHitRate2ChaosHits_value,featureHitRate2AllyHits_value,featureHitRate2HeroHits_value,featureHitRate2EnemyHits_value,featureHitRate2ShieldHits_value,featureHitRate2SummonerHits_value,featureSpecialData1Total_value,featureSpecialData1Value_value,ranked_value,rankedDelta_value
4,5513730_1765074019_1318548_11001_12,VP Sypeh,0,1,Invisible Woman,1050,Celestial Husk,Domination,381,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5308.0,13.0
13,6711722_1764888815_1272257_11001_22,VP Sypeh,0,1,Invisible Woman,1050,Birnin T'Challa,Domination,537,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5295.0,13.0
32,6711647_1764886525_1245250_11001_22,VP Sypeh,1,1,Invisible Woman,1050,Spider-Islands,Convoy,1015,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5281.0,16.0
43,506241_1764884213_1311009_11001_21,VP Sypeh,0,1,Gambit,1058,Arakko,Convoy,1187,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5264.0,15.0
48,506105_1764882596_1310017_11001_21,VP Sypeh,1,0,Invisible Woman,1050,Krakoa,Domination,883,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5248.0,-23.0
71,5513709_1764826259_2042189_11001_12,VP Sypeh,0,1,Gambit,1058,Heart Of Heaven,Convergence,234,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5271.0,11.0
74,5513562_1764654765_1291214_11001_12,VP Sypeh,1,1,Invisible Woman,1050,Midtown,Convoy,960,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5259.0,15.0
88,5513669_1764653691_1231370_11001_12,VP Sypeh,1,1,Rocket Raccoon,1023,Yggdrasill Path,Convoy,259,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5244.0,13.0
105,5513547_1764535377_1311217_11001_12,VP Sypeh,1,1,Gambit,1058,Arakko,Convoy,749,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5230.0,17.0
117,5513459_1764389811_1291129_11001_12,VP Sypeh,0,1,Rocket Raccoon,1023,Midtown,Convoy,639,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5213.0,16.0


i will select for specific columns, 72 is a bit much, need to see what the fluff is

In [8]:
matches_cleaned.head().to_csv('matches_cleaned_head.csv', index=False)

I looked through the file, found columns that are meaningless for me, and copy pasted below, and put into list to drop

I dont need any of these columns: 
sessionHitRate_value,maxSurvivalKills_value,maxContinueKills_value,maxContinueWins_value,continueKills3_value,continueKills4_value,continueKills5_value,continueKills6_value,mainAttacks_value,mainAttackHits_value,shieldHits_value,summonerHits_value,chaosHits_value,heroSessionHitRate_value,heroMainAttacks_value,heroMainAttackHits_value,heroShieldHits_value,heroSummonerHits_value,heroChaosHits_value,featureNormalData1_value,featureNormalData2_value,featureNormalData3_value,featureCriticalRate1CritHits_value,featureCriticalRate1Hits_value,featureCriticalRate2CritHits_value,featureCriticalRate2Hits_value,featureHitRate1UseCount_value,featureHitRate1RealHitHeroCount_value,featureHitRate1ChaosHits_value,featureHitRate1AllyHits_value,featureHitRate1HeroHits_value,featureHitRate1EnemyHits_value,featureHitRate1ShieldHits_value,featureHitRate1SummonerHits_value,featureHitRate2UseCount_value,featureHitRate2RealHitHeroCount_value,featureHitRate2ChaosHits_value,featureHitRate2AllyHits_value,featureHitRate2HeroHits_value,featureHitRate2EnemyHits_value,featureHitRate2ShieldHits_value,featureHitRate2SummonerHits_value,featureSpecialData1Total_value,featureSpecialData1Value_value,ranked_value,rankedDelta_value

In [9]:
cols_to_drop = [
    "sessionHitRate_value","maxSurvivalKills_value","maxContinueKills_value",
    "maxContinueWins_value","continueKills3_value","continueKills4_value",
    "continueKills5_value","continueKills6_value","mainAttacks_value",
    "mainAttackHits_value","shieldHits_value","summonerHits_value",
    "chaosHits_value","heroSessionHitRate_value","heroMainAttacks_value",
    "heroMainAttackHits_value","heroShieldHits_value","heroSummonerHits_value",
    "heroChaosHits_value","featureNormalData1_value","featureNormalData2_value",
    "featureNormalData3_value","featureCriticalRate1CritHits_value",
    "featureCriticalRate1Hits_value","featureCriticalRate2CritHits_value",
    "featureCriticalRate2Hits_value","featureHitRate1UseCount_value",
    "featureHitRate1RealHitHeroCount_value","featureHitRate1ChaosHits_value",
    "featureHitRate1AllyHits_value","featureHitRate1HeroHits_value",
    "featureHitRate1EnemyHits_value","featureHitRate1ShieldHits_value",
    "featureHitRate1SummonerHits_value","featureHitRate2UseCount_value",
    "featureHitRate2RealHitHeroCount_value","featureHitRate2ChaosHits_value",
    "featureHitRate2AllyHits_value","featureHitRate2HeroHits_value",
    "featureHitRate2EnemyHits_value","featureHitRate2ShieldHits_value",
    "featureHitRate2SummonerHits_value","featureSpecialData1Total_value",
    "featureSpecialData1Value_value","ranked_value","rankedDelta_value"
]

matches_cleaned = matches_cleaned.drop(columns=cols_to_drop)
matches_cleaned

,match_id,player_id,team_id,won,hero_name,hero_id,map_name,match_mode,duration_sec,is_mvp,...,kdaRatio_value,totalHeroDamage_value,totalHeroDamagePerMinute_value,totalHeroHeal_value,totalHeroHealPerMinute_value,totalDamageTaken_value,totalDamageTakenPerMinute_value,lastKills_value,headKills_value,soloKills_value
0,5513730_1765074019_1318548_11001_12,Deleted404,0,1,Rocket Raccoon,1023,Celestial Husk,Domination,381,False,...,10.000000,655.769723,109.29,11943.727504,1990.62,2981.606651,496.93,2,0,0
1,5513730_1765074019_1318548_11001_12,Kaifs,0,1,Magneto,1037,Celestial Husk,Domination,381,False,...,13.000000,7781.313597,1296.89,0.000000,0.00,11832.338579,1972.06,4,0,1
2,5513730_1765074019_1318548_11001_12,Finnsi,0,1,Groot,1027,Celestial Husk,Domination,381,False,...,6.500000,8685.591116,1447.60,0.000000,0.00,19402.304778,3233.72,6,0,1
3,5513730_1765074019_1318548_11001_12,JukeBox2274,1,0,Angela,1056,Celestial Husk,Domination,381,False,...,0.666667,6113.298075,1018.88,0.000000,0.00,18688.655201,3114.78,1,0,1
4,5513730_1765074019_1318548_11001_12,VP Sypeh,0,1,Invisible Woman,1050,Celestial Husk,Domination,381,False,...,27.000000,4548.798798,758.13,14847.611123,2474.60,6147.800565,1024.63,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64051,5513925_1765094719_1231224_11001_12,iffvy,1,0,Winter Soldier,1041,Yggdrasill Path,Convoy,1059,False,...,3.818182,32969.463813,1939.38,2477.660993,145.74,19305.615006,1135.62,24,0,1
64052,5513925_1765094719_1231224_11001_12,Rajahyrax,1,0,Groot,1027,Yggdrasill Path,Convoy,1059,False,...,4.555556,28770.961880,1692.41,0.000000,0.00,82469.576583,4851.15,7,0,1
64053,5513925_1765094719_1231224_11001_12,Пyк,1,0,Rocket Raccoon,1023,Yggdrasill Path,Convoy,1059,False,...,7.285714,5906.691635,347.45,39778.870284,2339.93,12175.875208,716.23,1,0,0
64054,5513925_1765094719_1231224_11001_12,FAZE BLAZIKEN,0,1,Phoenix,1054,Yggdrasill Path,Convoy,1059,False,...,3.200000,38203.973442,2247.29,1665.143768,97.95,15347.804127,902.81,13,1,2


Thought process of analyzing team compositions: 
Group by match id, then separate into each team, and which team won. 
Then can do like a large scale analysis of repeated teams and win rates

In [10]:
matches_cleaned.groupby(["hero_name", "map_name"])["won"].mean()


hero_name     map_name         
Adam Warlock  Arakko               0.419355
              Birnin T'Challa      0.473684
              Celestial Husk       0.529412
              Central Park         0.500000
              Hall Of Djalia       0.500000
                                     ...   
Wolverine     Midtown              0.642857
              Royal Palace         0.000000
              Spider-Islands       0.558140
              Symbiotic Surface    0.455882
              Yggdrasill Path      0.459016
Name: won, Length: 539, dtype: float64

In [11]:
matches_cleaned.groupby(["hero_name", "match_mode"])["won"].mean()


hero_name       match_mode 
Adam Warlock    Convergence    0.461538
                Convoy         0.500000
                Domination     0.438596
Angela          Convergence    0.555556
                Convoy         0.550256
                                 ...   
Winter Soldier  Convoy         0.427657
                Domination     0.446271
Wolverine       Convergence    0.535055
                Convoy         0.557927
                Domination     0.542088
Name: won, Length: 132, dtype: float64

In [12]:
matches_cleaned[matches_cleaned.player_id == 'VP Sypeh'].groupby(["hero_name", "match_mode"])["won"].mean()


hero_name        match_mode 
Adam Warlock     Domination     1.000000
Emma Frost       Convoy         1.000000
Gambit           Convergence    0.600000
                 Convoy         0.857143
                 Domination     0.500000
Invisible Woman  Convergence    0.666667
                 Convoy         1.000000
                 Domination     0.714286
Loki             Convergence    1.000000
                 Convoy         1.000000
                 Domination     1.000000
Mantis           Convoy         0.000000
Rocket Raccoon   Convoy         1.000000
Name: won, dtype: float64

the above outcomes match what i see in the per player data, so this is good (to double check the data is valid/reliable in case of discrepancies)

Thought process:  

What i want to analyze with this data set is the team compositions per team (which combination of 6 heroes are there per team) and who won?   

Based on that I want to create like a matchup criteria, where if theres one team composition, how likely are they to win against an enemy team composition.   

And also need to see, if the enemy team composition is unknown, how good is the single team composition's performance in general?

In [13]:
# gonna get specific team comps now into their own rows
team_row_separating = matches_cleaned[['match_id', 'team_id', 'hero_name', 'won']].copy()
team_row_separating

,match_id,team_id,hero_name,won
0,5513730_1765074019_1318548_11001_12,0,Rocket Raccoon,1
1,5513730_1765074019_1318548_11001_12,0,Magneto,1
2,5513730_1765074019_1318548_11001_12,0,Groot,1
3,5513730_1765074019_1318548_11001_12,1,Angela,0
4,5513730_1765074019_1318548_11001_12,0,Invisible Woman,1
...,...,...,...,...
64051,5513925_1765094719_1231224_11001_12,1,Winter Soldier,0
64052,5513925_1765094719_1231224_11001_12,1,Groot,0
64053,5513925_1765094719_1231224_11001_12,1,Rocket Raccoon,0
64054,5513925_1765094719_1231224_11001_12,0,Phoenix,1


grouping them up now

In [14]:
#have to sort during the lambda function otherwise the same heroes in different orders will be treated as different comps
#grouping by the match and the 2 teams, and then for each team aggregating the heroes and ordering them w/ comma separation
team_comps = team_row_separating.groupby(['match_id', 'team_id']).agg({'hero_name': lambda x: ', '.join(sorted(x)),
'won': 'max'}) 
team_comps = team_comps.reset_index()
team_comps

,match_id,team_id,hero_name,won
0,4859912_1764413878_2042276_11001_32,0,"Gambit, Invisible Woman, Magneto, Phoenix, Spi...",1
1,4859912_1764413878_2042276_11001_32,1,"Gambit, Invisible Woman, Magneto, Spider-Man, ...",0
2,4859918_1764785795_1291050_11001_32,0,"Daredevil, Gambit, Luna Snow, Magneto, Venom, ...",0
3,4859918_1764785795_1291050_11001_32,1,"Gambit, Iron Fist, Magneto, Phoenix, Rocket Ra...",1
4,4859927_1764258769_1291106_11001_32,0,"Blade, Emma Frost, Invisible Woman, Magneto, R...",1
...,...,...,...,...
10671,6713333_1763758119_1267033_11001_22,1,"Black Widow, Gambit, Invisible Woman, Magneto,...",1
10672,6713340_1764966783_2042019_11001_22,0,"Black Panther, Emma Frost, Invisible Woman, Lu...",0
10673,6713340_1764966783_2042019_11001_22,1,"Angela, Doctor Strange, Hawkeye, Invisible Wom...",1
10674,6713853_1736615457_110_11001_10,0,"Invisible Woman, Iron Man, Mantis, Mister Fant...",1


each match id is listed 2 times (one row for each team comp)

In [15]:
team_comps.to_csv('team_comps.csv', index=False)

now that the matches have the set of heroes on each team, I want to see what the win rate in general of each team composition, with each team composition being the set of heroes under the 'hero_name' column

In [16]:
win_rates = team_comps.groupby('hero_name')['won'].mean().reset_index()
win_rates = win_rates.rename(columns={'won': 'win_rate'})
win_rates

,hero_name,win_rate
0,"Adam Warlock, Adam Warlock, Bruce Banner, Gamb...",0.0
1,"Adam Warlock, Angela, Bruce Banner, Gambit, In...",1.0
2,"Adam Warlock, Angela, Captain America, Cloak &...",0.0
3,"Adam Warlock, Angela, Captain America, Daredev...",0.0
4,"Adam Warlock, Angela, Captain America, Hela, J...",0.0
...,...,...
7834,"Magneto, Moon Knight, Psylocke, Rocket Raccoon...",1.0
7835,"Magneto, Peni Parker, Phoenix, Psylocke, Rocke...",0.0
7836,"Magneto, Phoenix, Rocket Raccoon, Spider-Man, ...",0.0
7837,"Mantis, Psylocke, Rocket Raccoon, Thor, Venom,...",1.0


the above doesnt look right, is the list of hero names is being treated like an ordered list instead of a set?  

How come theres 7839 unique team compositions? I find that wierd

In [17]:
win_rates.to_csv('team_comp_win_rates.csv', index=False)

Actually, looking through the csv file, it actually does look right. It is very crazy yto see that there can be so many unique team compositions, but it is unsurprising when considering the total #heroes in the game

The math does check out though, as there are currently 44 heroes in the game, so the total combinations of heroes possible per team is a pretty high number. 

I want the total count a team composition appears in the table so i know whether 100% winrate is from 30 matches or 1

In [18]:
counts = team_comps['hero_name'].value_counts().reset_index()
counts.columns = ['hero_name', 'game_count']
win_rates = win_rates.merge(counts, on='hero_name')
win_rates

,hero_name,win_rate,game_count
0,"Adam Warlock, Adam Warlock, Bruce Banner, Gamb...",0.0,1
1,"Adam Warlock, Angela, Bruce Banner, Gambit, In...",1.0,1
2,"Adam Warlock, Angela, Captain America, Cloak &...",0.0,1
3,"Adam Warlock, Angela, Captain America, Daredev...",0.0,1
4,"Adam Warlock, Angela, Captain America, Hela, J...",0.0,1
...,...,...,...
7834,"Magneto, Moon Knight, Psylocke, Rocket Raccoon...",1.0,1
7835,"Magneto, Peni Parker, Phoenix, Psylocke, Rocke...",0.0,1
7836,"Magneto, Phoenix, Rocket Raccoon, Spider-Man, ...",0.0,1
7837,"Mantis, Psylocke, Rocket Raccoon, Thor, Venom,...",1.0,1


In [19]:
matches_cleaned.to_csv('matches_cleaned.csv', index=False)
win_rates.to_csv('team_comp_win_rates.csv', index=False)